# Assignment One - Analytics for Unstructured Data
#### Team Members: Haden Loveridge, Mikala Lowrance, Arantza Garcia Delfin, Neha R Boinapalli, Lindsay Bartol, Sameer Kahn, Biagio Alessandrello 

#### - 10 Brands we chose - Frequency Table

#### - Show all lift values in a table

#### - MDS Map

#### - State the 5 attributes you chose (again, a frequency table is good here)

#### - For task F, provide all details of your analysis - e.g. how you measured "aspirational" and how you found the most aspirational brand.

#### - Advice/Insights based on your analysis for your client

#### --------------- Code ---------------

#### - Scraper

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup


def scrape_page(page_number):
    url = f"https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p{page_number}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='vanilla_discussion_index')
    print(f"-- Scraping {url}")

    # GET ALL THE COMMENTS STARTING FROM THE LATEST
    forum_posts = results.find_all("div", class_="Comment")
    forum_posts.reverse()

    rows = []
    for post in forum_posts:
        post_date = post.find("span", class_="MItem DateCreated")
        post_comment = post.find("div", class_="Message userContent")

        # REMOVE QUOTED REPLIES, IF THEY EXIST
        if post_comment.find("blockquote"):
            post_comment.find("blockquote").decompose()

        row = [post_date.text.strip(), post_comment.text.strip()]
        rows.append(row)

    print(f"---- Got {len(rows)} comments")
    return rows


def scrape_pages(page_number=435, target_comment_count=5000):
    comments = []

    while len(comments) <= target_comment_count:
        print(f"- Total comments so far: {len(comments)}")
        page_data = scrape_page(page_number)
        comments.extend(page_data)
        page_number -= 1

    return comments


def print_csv(rows, filename='edmunds_comments'):
    with open(f'{filename}.csv', 'w', newline='') as file:
        writer = csv.writer(file)

        writer.writerow(["Date", "Comment"])
        writer.writerows(rows)


def run():
    data = scrape_pages()
    print_csv(data)


### - Task A

In [ ]:
import csv
import re
import string
from collections import defaultdict
from nltk.corpus import stopwords
import nltk
import io

#gets rid of word frequency definition
from collections import Counter

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# Input and output filenames
input_filename = 'edmunds_comments.csv' # Input file
final_filename = 'A_mid.csv' # Intermediate file without the column header
word_freq_output = 'A_word_freq.csv' # Output file for word frequencies

In [ ]:
# Function to clean and tokenize sentences
def clean_and_tokenize(sentence):
    """
    Cleans a given sentence by removing punctuation and stopwords, converting text
    to lowercase,
    and tokenizing the remaining words.
    """
    # Remove punctuation and convert text to lowercase
    sentence = re.sub(f'[{re.escape(string.punctuation)}]', '', sentence.lower())
    # Tokenize and remove stopwords
    return sentence.split()

In [ ]:
# Step 1: Remove header from the input CSV and create a new file without it
def remove_header(input_file, output_file):
    """
    Reads the input CSV file, removes the header, and writes the remaining rows
    into a new output file.
    """
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        next(infile) #skips the header
        outfile.writelines(infile)

In [ ]:
#OUR SAMPLE FILE HAS THE COMMENTS IN THE SECOND COLUMN NOT THE THIRD

In [ ]:
# Step 2: Extract and clean sentences from the text
def extract_sentences(file):
    """
    Extracts text data from the second column of the CSV file, splits it into
    sentences,
    and cleans each sentence by removing punctuation and stopwords.
    """

    sentences_clean = []

    with open(file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        
        # Iterate over rows in the CSV file
        for row in reader:
            # Assuming the text is in the third column (index 1)
            if len(row) > 1:
                text = row[1]
                
                # Split the text into sentences based on punctuation (.?!)
                sentences = re.split(r'[.!?]', text)
                
                # Clean and tokenize each sentence
                for sentence in sentences:
                    cleaned_tokens = clean_and_tokenize(sentence)
                    if cleaned_tokens:  # Avoid adding empty sentences
                        sentences_clean.extend(cleaned_tokens)
                        #sentences_clean.append(cleaned_tokens)
    return sentences_clean

In [ ]:
# Step 4: Write word frequencies to CSV
def write_word_frequencies(word_freq, output_file):
    """
    Writes the word frequencies to the specified CSV file.
    """
    # Sort the word frequencies by frequency in descending order
    sorted_word_freq = sorted(word_freq.items(), key=lambda item: item[1], reverse=True)
    
    with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Word', 'Frequency'])  # Write header row

        for word, frequency in sorted_word_freq:
            writer.writerow([word, frequency])
    
    print(f"Word frequencies written to {output_file}")

In [ ]:
# Main function to execute the workflow
def main():
    # Step 1: Remove header from the input CSV and create a new file without it
    remove_header(input_filename, final_filename)

    # Step 2: Extract and clean sentences from the text
    sentences_clean = extract_sentences(final_filename)

    # Step 3: Calculate word frequencies
    word_frequencies = Counter(sentences_clean)
    #calculate_word_frequencies(sentences_clean)

    # Step 4: Write word frequencies to CSV
    write_word_frequencies(word_frequencies, word_freq_output)

In [ ]:
# Run the main function
if __name__ == "__main__":
    main()

##### Economic Testing

In [ ]:
df = pd.read_csv('A_word_freq.csv')

df['Rank'] = df.index + 1

df['log_rank'] = np.log(df['Rank'])
df['log_freq'] = np.log(df['Frequency'])

df[:3]

In [ ]:
X = sm.add_constant(df['log_rank'])

# Create the OLS model
model = sm.OLS(df['log_freq'], X)

# Fit the model
results = model.fit()

In [ ]:
intercept = results.params['const']  # Intercept (the constant)
slope = results.params['log_rank']          # Slope (coefficient for x)

# Extract the R-squared value
r_squared = results.rsquared

# Extract the p-value for the slope
p_value = results.pvalues['log_rank']

conf_intervals = results.conf_int()

t_value = results.tvalues['log_rank']

In [ ]:
# Print the results
print(f"Slope (Zipf's Law exponent s): {slope}")
print(f"Intercept: {intercept}")
print(f"R-squared: {r_squared}")
print(f"P-value: {p_value}")
print(f"T-value: {t_value}")
print(f"Confidence Intervals:\n{conf_intervals}")

##### Top 100 Graph

In [ ]:
df_100 = df[:100].copy()

In [ ]:
log_rank_end = df_100['log_rank'].iloc[99]  # log rank at rank 100
log_freq_end = df_100['log_freq'].iloc[99]
# Normalize all log frequencies by the log frequency at rank 100
df_100['norm_log_freq'] = df_100['log_freq'] - log_freq_end

In [ ]:
X4 = sm.add_constant(df_100['log_rank'])

# Create the OLS model
model4 = sm.OLS(df_100['norm_log_freq'], X4)

# Fit the model
results4 = model4.fit()

intercept4 = results4.params['const']  # Intercept (the constant)
slope4 = results4.params['log_rank']  

print(f"Slope (Zipf's Law Slope s): {slope4}")
print(f"Intercept: {intercept4}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_100['log_rank'], df_100['norm_log_freq'], label='Data', color='blue')

#plt.plot(df_100['log_rank'], intercept1 + slope1 * df_100['log_rank'], 'r', label='Fit: f(r) ∝ 1/r^s')


x_vals = df_100['log_rank']
y_vals = df_100['norm_log_freq'].iloc[-1] - (x_vals - log_rank_end)  # since slope is -1

plt.plot(x_vals, y_vals, 'g--', label=f'Theoretical Slope = -1')

plt.xlabel('Log Rank')
plt.ylabel('Log Normalized Frequency')
plt.title('Zipf\'s Law Fit for Most Common 100 Words: Empirical vs Theoretical Slope')
plt.legend()
plt.grid(True)
plt.show()

### - Task B

#### Word Frequencies (with car replacement, no stop words, by cell/text)

In [ ]:
import csv
import re
import string
from collections import defaultdict
from nltk.corpus import stopwords
import nltk
import io
import pandas as pd
import math
import numpy as np

In [ ]:
#defining variables that might change when I switch from sample data to scraped data

data = 'edmunds_comments.csv'
text_row = 2 - 1
output_file = 'B_word_freq.csv'

In [ ]:
#create a dictionary from the car brand and model csv

df = pd.read_csv('car_models_and_brands.csv')
model_to_brand = dict(zip(df['Model'], df['Brand']))

In [ ]:
#function to clean and tokenize - here I'm taking out stop words and replacing model with brand as well

#Download and initialize stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_and_tokenize (sentence):
    #first we're going to remove punctuation
    no_punct = re.sub(f'[{re.escape(string.punctuation)}]', ' ', sentence)
    #now make lowercase
    lowered = no_punct.lower()
    #take out possessive punctuation
    for original, replacement in model_to_brand.items():
        lowered = re.sub(rf'\b{re.escape(original)}\'?s?\b', replacement, lowered, flags=re.IGNORECASE)
    #split the sentence into words
    words = lowered.split()
    #replace car models with brand names and remove stop words
    cleaned_words = []
    for word in words:
        if word in model_to_brand:
            cleaned_words.append(model_to_brand[word])  # Replace model with brand
        elif word not in stop_words:
            cleaned_words.append(word)  # Keep the word if it's not a stop word
    return cleaned_words

In [ ]:
#Function to remove header, but I'm not actually using it

def remove_header(input, output):
    with open(input, 'r', newline = '') as header_file:
        reader = csv.reader(header_file)
        next(reader)
        #Now let's write the output file
        with open(output, 'w', newline='') as no_header_file:
            writer = csv.writer(no_header_file)
            for row in reader:
                writer.writerow(row)

In [ ]:
#function to extract posts and clean them

def extract_text(file):

    text_clean = []
    with open(file, 'r', newline='', encoding='utf-8') as data_file:
        reader = csv.reader(data_file)
        # Skip the header
        next(reader)
        for row in reader:
            text = row[text_row]
            cleaned_tokens = clean_and_tokenize(text)
            if cleaned_tokens:  # Avoid adding empty sentences
                text_clean.append(cleaned_tokens)
    return text_clean

In [ ]:
#Function to calculate word frequencies; counted once per post

def calculate_word_frequencies(text_list):
    frequencies = {}
    for text in text_list:
        unique_words = set(text)
        for word in unique_words:
            if word in frequencies:
                frequencies[word] += 1
            else:
                frequencies[word] = 1       
    return frequencies

In [ ]:
#now write the frequencies to a csv

def write_word_frequencies(word_freq, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as freq_file:
        writer = csv.writer(freq_file)
        #Write the header row
        writer.writerow(['Word', 'Frequency'])
        for word, freq in word_freq.items():
            writer.writerow([word, freq])        

In [ ]:
def main():
    cleaned_text = extract_text(data)
    freq_dict = calculate_word_frequencies(cleaned_text)
    write_word_frequencies(freq_dict, output_file)

In [ ]:
# Run the main function
if __name__ == "__main__":
    main()

##### Find Top Ten Brands

In [ ]:
df = pd.read_csv(output_file)
df

In [ ]:
df = pd.read_csv(output_file)
df_brands = pd.read_csv('car_models_and_brands.csv')

#let's get just the brands so we can filter the words df
df_brands = df_brands[df_brands['Brand'] != 'car']
df_brands = df_brands[df_brands['Brand'] != 'seat']
df_brands = df_brands[df_brands['Brand'] != 'sedan']
df_brands = df_brands[df_brands['Brand'] != 'problem']
df_brands = df_brands['Brand'].unique()

brand_list = list(df_brand_freq['Word'])
brand_list

#no filter for brand frequency
df_brand_freq = df[df['Word'].isin(df_brands)]
df_brand_freq = df_brand_freq.sort_values(by = 'Frequency', ascending = False)[:10].reset_index(drop = True)
df_brand_freq

### - Task C

In [ ]:
import pandas as pd
import string
import re
import itertools

# Read the CSV file into a dataframe
df = pd.read_csv('edmunds_comments.csv')

# Load the car models and brands CSV into a dataframe
df_brands = pd.read_csv('car_models_and_brands.csv')

# Create a dictionary for faster lookup, where keys are the models and values are the brands
model_to_brand = dict(zip(car_brands_df['Model'], car_brands_df['Brand']))

df['Comment'] = df['Comment'].astype(str)

##########################################
######### PREPROCESSING FUNCTION #########
##########################################

def preprocess_Comment(Comment):
    if isinstance(Comment, str):  # Only process if Comment is a string
        Comment = Comment.lower()  # convert to lowercase
        Comment = Comment.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))  # replace punctuation with spaces
        Comment = ' '.join(Comment.split())  # remove extra spaces
    return Comment

# Apply preprocessing to the 'Comment' column
df['Comment'] = df['Comment'].apply(preprocess_Comment)

##########################################
######### REPLACE MODEL NAMES ###########
##########################################

# Function to replace model names in the 'Comment' column with their associated brands
# Ensures the model name is replaced only if it's a standalone word
def replace_model_with_brand(Comment):
    if isinstance(Comment, str):  # Only process if Comment is a string
        for model, brand in model_to_brand.items():
            # Use word boundaries (\b) to ensure the model name is not part of a longer word
            Comment = re.sub(r'\b{}\b'.format(re.escape(model.lower())), brand.lower(), Comment)
    return Comment

# Apply the function to the 'Comment' column after preprocessing
df['Comment'] = df['Comment'].apply(replace_model_with_brand)

In [ ]:
##################################
######## CALCULATE LIFT ##########
##################################

brand_list = [brand.lower() for brand in brand_list]

def brand_mentioned(Comment, brand):
    # Use regular expressions to find whole words, ignoring punctuation
    return bool(re.search(rf'\b{re.escape(brand)}\b', Comment))

# Function to check if two brands are mentioned within 15 words of each other
def brands_within_15_words(Comment, brand1, brand2):
    words = Comment.split()
    brand1_indices = [i for i, word in enumerate(words) if word == brand1]
    brand2_indices = [i for i, word in enumerate(words) if word == brand2]
    for i in brand1_indices:
        for j in brand2_indices:
            if abs(i - j) <= 15:  # Check if brands are within 7 words
                return True
    return False

# Dictionary to store the lift values
lift_values = {}

# Calculate lift for each pair of brands
total_Comments = len(df)

for brand1, brand2 in itertools.combinations(brand_list, 2):
    # Count the number of Comments that mention brand1 and brand2 within 15 words of each other
    Comments_with_both = df['Comment'].apply(lambda Comment: brands_within_15_words(Comment, brand1, brand2)).sum()
    
    # Count the number of Comments that mention each brand (at least once)
    Comments_with_brand1 = df['Comment'].apply(lambda Comment: brand_mentioned(Comment, brand1)).sum()
    Comments_with_brand2 = df['Comment'].apply(lambda Comment: brand_mentioned(Comment, brand2)).sum()

    # print(f"Comments with {brand1}: {Comments_with_brand1}")
    # print(f"Comments with {brand2}: {Comments_with_brand2}")
    # print(f"Comments with {brand1} and {brand2}: {Comments_with_both}")
    # print()
    
    # Calculate the lift
    if Comments_with_brand1 > 0 and Comments_with_brand2 > 0:
        lift = (total_Comments * Comments_with_both) / (Comments_with_brand1 * Comments_with_brand2)
        lift_values[f'{brand1}-{brand2}'] = lift

In [ ]:
# # Sort the dictionary by lift values (smallest to largest)
# sorted_lift_values = dict(sorted(lift_values.items(), key=lambda item: item[1]))

# # Print the sorted lift values
# for brand_pair, lift in sorted_lift_values.items():
#     print(f'{brand_pair}: {lift}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Create an empty DataFrame for the lift values, with brands as rows and columns
lift_df = pd.DataFrame(index=brand_list, columns=brand_list)

# Populate the DataFrame with lift values
for key, lift_value in lift_values.items():
    brand1, brand2 = key.split('-')  # Split the key into brand1 and brand2
    lift_df.loc[brand1, brand2] = lift_value
    lift_df.loc[brand2, brand1] = lift_value  # Since lift is symmetric

# Fill the diagonal with NaN or 0 since a brand compared to itself isn't meaningful
np.fill_diagonal(lift_df.values, np.nan)

# Convert to numeric type
lift_df = lift_df.astype(float)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(lift_df, dtype=bool))

# Define a custom colormap that shifts from red (<1) to blue (>1) with a gradient scale
cmap = LinearSegmentedColormap.from_list('custom_cmap', ['red', 'white', 'blue'], N=256)

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(lift_df, annot=True, fmt=".3f", cmap=cmap, linewidths=.5, mask=mask, cbar_kws={'label': 'Lift Value'}, center=1)
plt.title("Lift Values Between Brands")
plt.show()


### - Task D

In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text


distance_matrix = np.zeros((len(brand_list), len(brand_list)))

for i, brand1 in enumerate(brand_list):
    for j, brand2 in enumerate(brand_list):
        if i != j:
            # Get the lift value if it exists, otherwise use a default high distance (e.g., 1e6)
            lift_value = lift_values.get(f'{brand1}-{brand2}') or lift_values.get(f'{brand2}-{brand1}') or 1e6
            distance_matrix[i, j] = 1 / lift_value  # Inverse of lift as distance

# Apply Multi-Dimensional Scaling (MDS)
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
mds_coordinates = mds.fit_transform(distance_matrix)

# Plot the results
plt.figure(figsize=(12, 8))

# Scatter plot with markers for each brand
plt.scatter(mds_coordinates[:, 0], mds_coordinates[:, 1], s=100, color='blue')

# List to store text objects for adjustment
texts = []

# Annotate points with brand names, and store text objects for adjustment
for i, brand in enumerate(brand_list):
    texts.append(plt.text(mds_coordinates[i, 0], mds_coordinates[i, 1], brand, fontsize=12, ha='right',
                          bbox=dict(facecolor='white', alpha=0.7)))

# Adjust the positions of the text labels to avoid overlap, add arrows pointing to the actual points
adjust_text(texts, x=mds_coordinates[:, 0], y=mds_coordinates[:, 1],
            arrowprops=dict(arrowstyle="->", color='gray', lw=1))

# Add title and axis labels
plt.title("MDS Map of Brands Based on Lift", fontsize=16)
plt.xlabel("MDS Dimension 1", fontsize=14)
plt.ylabel("MDS Dimension 2", fontsize=14)

# Add grid for better readability
plt.grid(True)

plt.show()

In [ ]:
print(f"Stress: {mds.stress_}")


In [ ]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Required for 3D plots

# Create the distance matrix
distance_matrix = np.zeros((len(brand_list), len(brand_list)))

for i, brand1 in enumerate(brand_list):
    for j, brand2 in enumerate(brand_list):
        if i != j:
            # Get the lift value if it exists, otherwise use a default high distance (e.g., 1e6)
            lift_value = lift_values.get(f'{brand1}-{brand2}') or lift_values.get(f'{brand2}-{brand1}') or 1e6
            distance_matrix[i, j] = 1 / lift_value  # Inverse of lift as distance

# Apply Multi-Dimensional Scaling (MDS) with 3 components
mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42)
mds_coordinates = mds.fit_transform(distance_matrix)

# Plot the results in 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with markers for each brand in 3D
ax.scatter(mds_coordinates[:, 0], mds_coordinates[:, 1], mds_coordinates[:, 2], s=100, color='blue')

# Annotate points with brand names
for i, brand in enumerate(brand_list):
    ax.text(mds_coordinates[i, 0], mds_coordinates[i, 1], mds_coordinates[i, 2], brand, fontsize=12)

# Add title and axis labels
ax.set_title("3D MDS Map of Brands Based on Lift", fontsize=16)
ax.set_xlabel("MDS Dimension 1", fontsize=14)
ax.set_ylabel("MDS Dimension 2", fontsize=14)
ax.set_zlabel("MDS Dimension 3", fontsize=14)

# Show the plot
plt.show()


### - Task E

### - Task F

### - Task G

### - Task H